In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from PIL import Image
import os
import cv2

def get_rectangle(mask, th=0.1):
    thres = th*255
    mask[mask>thres]=255
    mask[mask<=thres]=0
    
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)                                                                                           
    area = 0 
    
    x = -1; y = -1
    for cnt in contours:
        (x_,y_,w_,h_) = cv2.boundingRect(cnt)
        area_ = np.sum(mask[y_:y_+h_, x_:x_+w_])
        if area_ > area:
            x=x_; y=y_; w=w_; h=h_;
            area = area_
    
    rectimg = np.zeros_like(mask)
    if x != -1:
        rectimg[y:y+h, x:x+w] = 1
    return rectimg

def browse_images(predsfiles1, imgsfiles1, flowsfiles1, annotsfiles, predsfiles2):
    n = len(predsfiles1)
    def view_image(i):
        plt.figure(figsize=(30, 10), dpi=80)
        predimg = Image.open(predsfiles1[i])
        plt.subplot(3, 3, 1)
        plt.axis('off')
        plt.imshow(predimg)
        plt.title('File: %s' % predsfiles1[i], loc='center')
        
        img = Image.open(imgsfiles1[i])
        plt.subplot(3, 3, 2)
        plt.axis('off')
        plt.imshow(img)
        
        flow = Image.open(flowsfiles1[i])
        plt.subplot(3, 3, 3)
        plt.axis('off')
        plt.imshow(flow)
        
        annot = Image.open(annotsfiles[i])
        plt.subplot(3, 3, 5)
        plt.axis('off')
        plt.imshow(annot)
        
        predimg2 = Image.open(predsfiles2[i])
        plt.subplot(3, 3, 4)
        plt.axis('off')
        plt.imshow(predimg2)
        plt.title('File: %s' % predsfiles2[i], loc='center')
        
        plt.subplot(3, 3, 7)
        rectimg = get_rectangle(np.array(predimg), th=0.3)
        plt.axis('off')
        plt.imshow(rectimg)
        
        plt.subplot(3, 3, 8)
        rectimg2 = get_rectangle(np.array(predimg2), th=0.1)
        plt.axis('off')
        plt.imshow(rectimg2)
        plt.show()
        
    interact(view_image, i=(0,n-1))

results1_dir = 'outputs/vis_final/MoCA/outputs/vis_final/MoCA/matnet_twostream_attention_warping_fixclip/MATNet_epoch15/'
results2_dir = '/home/menna/Code/MATNet_MotionBiased/output/multigpu_reproduce/MATNet_epoch16/'
flows_dir = '/local/riemann/home/msiam/MoCA_filtered2/FlowImages_gap1/'
imgs_dir = '/local/riemann/home/msiam/MoCA_filtered2/JPEGImages/'
annots_dir = '/local/riemann/home/msiam/MoCA_filtered2/Annotations/'
sequence = 'cuttlefish_5'#'devil_scorpionfish'

def get_files(results_dir, imgs_dir, flows_dir, annots_dir, sequence):
    preds_dir = os.path.join(results_dir, sequence)
    imgs_dir = os.path.join(imgs_dir, sequence)
    flows_dir = os.path.join(flows_dir, sequence)
    annots_dir = os.path.join(annots_dir, sequence)
    
    predsfiles = sorted([os.path.join(preds_dir, f) for f in os.listdir(preds_dir)])
    flowsfiles = sorted([os.path.join(flows_dir, f) for f in os.listdir(flows_dir)])
    imgsfiles = sorted([os.path.join(imgs_dir, f) for f in os.listdir(imgs_dir)])
    annotsfiles = sorted([os.path.join(annots_dir, f) for f in os.listdir(annots_dir)])
    return predsfiles, imgsfiles, flowsfiles, annotsfiles

predsfiles1, imgsfiles, flowsfiles, annotsfiles = get_files(results1_dir, imgs_dir, flows_dir, annots_dir, sequence)
predsfiles2, _, _, _ = get_files(results2_dir, imgs_dir, flows_dir, annots_dir, sequence)
browse_images(predsfiles1, imgsfiles, flowsfiles, annotsfiles, predsfiles2)


interactive(children=(IntSlider(value=11, description='i', max=22), Output()), _dom_classes=('widget-interact'…